## Data Collection Task

### Mô tả công việc:
1. Trang web cào dữ liệu:\
   a. [kokotaru](https://kokotaru.com/) - Tổng: ~ 500 Bài viết.\
   b. [kitchenart](https://cook.kitchenart.vn/cong-thuc-nau-an/) - Tổng: ~ 800 bài viết.
2. Dữ liệu cần cào:

   | Tên món ăn | Các thành phần chính |
   | -----------| ---------------------|

### Cài đặt thư viện

In [1]:
# !pip install bs4
# !pip install selenium
# !pip install numpy
# !pip install pandas
# !pip install requests
# !pip install tqdm

### Import các thư viện và các hàm dùng chung

In [2]:
from Libraries_Using import *
from Shared_functions import *

### **Nhận xét:**
Vì hai trang web này có cách hoạt động và tương tác khác nhau cho nên sẽ có 2 phần thu thập thông tin riêng biệt.
1. **Kokotaru:** Tương tác cuộn chuột để load thêm các bài viết.
2. **Kitchenart:** Phân ra các trang riêng biệt, mỗi trang gồm 20 bài viết.

### KOKOTARU WEBSITE PARSING

#### Tương tác:
* Load page theo thao tác cuộn chuột của người dùng.
#### Mô tả:
* Tất cả bài viết được tập hợp chung ở một link homepage của kokotaru.

### **Bước 1: Lấy ra nội dung của trang web bằng thư viện Selenium.**

* Định dạng đường dẫn cơ bản

In [3]:
KOKOTARU_BASE_URL = 'https://kokotaru.com/'

**Bước 1.1:** Tạo selenium chrome browser.\
**Bước 1.2:** Cho trang load hết tất cả nội dung.

In [4]:
def kokotaru_page_loader(BASE_URL: str = KOKOTARU_BASE_URL) -> str:
    
    driver = webdriver.Chrome()

    driver.get(BASE_URL)

    last_height = driver.execute_script("return document.body.scrollHeight")
    
    progress_bar = tqdm(desc="Scrolling", ncols=100, leave=True, unit="lần")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        new_height = driver.execute_script("return document.body.scrollHeight")
        
        progress_bar.update(1)
        
        if new_height == last_height:
            break
        
        last_height = new_height

    html_content = driver.page_source

    driver.quit()
    
    progress_bar.close() 
    print("Parsing completed.")
    
    return html_content

In [ ]:
kokotaru_html = kokotaru_page_loader(KOKOTARU_BASE_URL)

**Bước 1.3:** Lấy nội dung cho vào `kokotaru_html_content.html`

In [6]:
write_to_file("Assert/kokotaru_html_content.html",kokotaru_html, 'w_b_str')

#### **Bước 2: Lấy ra các link bài viết có trong homepage**

In [7]:
kokotaru_content = read_from_file("Assert/kokotaru_html_content.html", 'r_b_str')

In [8]:
def get_kokotaru_articles_urls(html_content) -> list:
    urls = []
    soup = BeautifulSoup(html_content, 'html.parser')
    article_headers = soup.find_all("article", { "class" : "entry-preview" })
    for header in article_headers:
        found = header.find('a', class_='cs-overlay-link')
        url = found['href'] if found else None
        if url is not None:
            urls.append(url)
    return urls

kokotaru_article_urls = get_kokotaru_articles_urls(kokotaru_content)


Ghi lại các link urls lấy được vào trong `kokotaru_aricle_urls.txt`

In [9]:
write_to_file('Assert/kokotaru_aricle_urls.txt', kokotaru_article_urls, 'w_b_element')